# Experiment on replicating `Modeling rain-driven overland flow`
The goal is to simulate rain accumulating on a surface and flowing, using the Poiseuille friction model which the [paper](https://arxiv.org/pdf/1609.04711) found most suitable for these conditions.

1. Geometry = a tilted rectangle, not a basin

In [1]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import *
from ngsolve.webgui import Draw


L  = 4.04 # flume length  [m]
W  = 0.115 # flume width   [m]
SlopeDeg = 5 # (%)  ➔  2 %  ⇒ 0.02
S0   = SlopeDeg/100 # tan α  (positive downslope)

# create a flat rectangle; the slope only appears in the source term
geo = Rectangle(L,W).Face()
geo.edges.Min(X).name="inlet"
geo.edges.Max(X).name="outlet"
geo.edges.Min(Y).name = "wall"
geo.edges.Max(Y).name = "wall"
geo = OCCGeometry(geo, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
# mesh.Curve(3)
Draw(mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

2. Unknowns, rainfall source and friction law

In [2]:
order = 2
fes   = L2(mesh,order=order)**3 # (h, h u, h v)

U,V = fes.TnT() # "Trial" and "Test" function
h, hu, hv = U

g  = 9.81 # [m/s²]
nu = 1e-6 # kinematic viscosity of water  [m²/s]
I_mm_h = 250 # rainfall intensity [mm h⁻¹]
R = I_mm_h/1000/3600 # convert to m s⁻¹  (≈ 6.94 × 10⁻⁶)
eps = 1e-6


In [3]:
def Sf_poiseuille(U):
    h, hu, hv = U
    qx = hu
    qy = hv
    return (3*nu)/g * CF( ( qx/h**3,  qy/h**3 ) )

In [4]:
n = specialcf.normal(mesh.dim)
def Max(a, b):
    """point-wise maximum for CoefficientFunctions"""
    return IfPos(a-b, a, b)

In [5]:
def F(U):
    h, hvx, hvy = U
    vx = hvx/h
    vy = hvy/h
    return CF(((hvx,hvy),
               (hvx*vx + 0.5*g*h**2, hvx*vy),
               (hvx*vy, hvy*vy + 0.5*g*h**2)),dims=(3,2))

In [6]:
n = specialcf.normal(mesh.dim)
def Max(u,v):
    return IfPos(u-v,u,v)
def Fmax(A,B): # max. characteristic speed:
    ha, hua, hva = A
    hb, hub, hvb = B
    vnorma = sqrt(hua**2+hva**2)/ha
    vnormb = sqrt(hub**2+hvb**2)/hb
    return Max(vnorma+sqrt(g*A[0]),vnormb+sqrt(g*B[0]))

def Fhatn(U): # numerical flux
    Uhat = U.Other(bnd=Ubnd)
    return (0.5*F(U)+0.5*F(Uhat))*n + Fmax(U,Uhat)/2*(U-Uhat)

In [7]:

U0 = CF((eps, 0, 0))
Ubnd = CF((h, -hu, -hv))

In [8]:
print(F(U).shape)
print(F(U).trans.shape)
print(Grad(V).shape)
print(Grad(V).trans.shape)


(3, 2)
(2, 3)
(3, 2)
(2, 3)


3. DG bilinear plus source terms (rain, bed-slope, friction)

In [9]:
def DGBilinearForm(fes,F,Fhatnd, Ubnd):
    a = BilinearForm(fes, nonassemble=True)
    a += - InnerProduct(F(U),Grad(V)) * dx
    a += InnerProduct(Fhatn(U),V) * dx(element_boundary=True)
    return a

a = DGBilinearForm(fes,F,Fhatn, Ubnd)

In [10]:
from DGdiffusion import AddArtificialDiffusion

artvisc = Parameter(1.0)
if order > 0:
    AddArtificialDiffusion(a,Ubnd,artvisc,compile=True)

In [11]:
invMA = fes.Mass(1).Inverse() @ a.mat


In [12]:
def Source(gfu):
    h  = gfu.components[0]
    hu = gfu.components[1]
    hv = gfu.components[2]
    vx = hu/h
    vy = hv/h

    # rainfall:  (R, 0, 0)
    srain_h  = R
    srain_hu = 0
    srain_hv = 0

    # bed-slope in +x direction:  +g h S₀
    sslope_hu = g*h*S0
    sslope_hv = 0

    friction = Sf_poiseuille((h, hu, hv))
    sfx = friction[0]
    sfy = friction[1]
    

    return CF( ( srain_h,
                 srain_hu + sslope_hu + sfx,
                 srain_hv + sslope_hv + sfy ) )

4. Time integrator

In [13]:
from math import ceil

eps = 1e-6
gfu = GridFunction(fes)
gfu.components[0].Set(eps)
tmp = gfu.vec.CreateVector()
tmp[:] = eps
gfu.vec.data += tmp # prevent division by 0

dt = 0.005
T  = 100


def TimeLoopExplicit(a, gfu, invMA,
                     dt, T,
                     SourceCF,
                     scenes=None,
                     nsamplings=100,
                     multidim_draw=False,
                     md_nsamplings=20):

    if scenes is None: scenes = [ Draw(gfu.components[0], mesh, "h", autoscale=True) ]

    res  = gfu.vec.CreateVector()      # residual  r  =  M⁻¹ A u
    rhs  = gfu.vec.CreateVector()      # source    S(u)
    gfs  = GridFunction(gfu.space)     # projection helper for S(u)

    if multidim_draw:
        gfu_t = GridFunction(gfu.space, multidim=True)
        gfu_t.AddMultiDimComponent(gfu.vec)

        nsteps = int(ceil(T/dt))
    t      = 0.0

    with TaskManager():
        for istep in range(1, nsteps+1):

            # 1) residual  r = M⁻¹ A u  (res ← invMA · u)
            invMA.Mult(gfu.vec, res)

            # 2) explicit source  rhs = S(u)
            gfs.Set( SourceCF(gfu) )   # project CF to FE space
            rhs[:] = gfs.vec           # copy into working vector

            # 3) forward Euler update:   u ← u − dt ( r − rhs )
            res -= rhs
            #gfu.vec.AddVector(-dt, res) 
            gfu.vec.data -= dt * res  

            t += dt

            if istep % max(1, nsteps//nsamplings) == 0:
                for s in scenes: s.Redraw()
            total_h = Integrate(gfu.components[0], mesh)
            if istep % 500 == 0:                     # every 5 s
                print(f"\nMean depth = {total_h / (L*W):.6f} m")

            if multidim_draw and istep % max(1, nsteps//md_nsamplings) == 0:
                gfu_t.AddMultiDimComponent(gfu.vec)

            print(f"\rt = {t:7.2f}  ({istep}/{nsteps})", end="", flush=True)

    for s in scenes: s.Redraw()

    if multidim_draw:
        return gfu_t


In [ ]:
scenes = [ Draw(gfu.components[0], mesh, "h") ]  

gfu_t = TimeLoopExplicit(a,
                         gfu       = gfu,
                         invMA     = invMA,
                         dt        = 0.005,
                         T         = 100,
                         SourceCF  = Source,
                         scenes    = scenes,
                         multidim_draw = True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

t =    2.49  (499/20000)
Mean depth = nan m
t =    4.99  (999/20000)
Mean depth = nan m
t =    7.49  (1499/20000)
Mean depth = nan m
t =   10.00  (1999/20000)
Mean depth = nan m
t =   12.50  (2499/20000)
Mean depth = nan m
t =   15.00  (2999/20000)
Mean depth = nan m
t =   17.50  (3499/20000)
Mean depth = nan m
t =   18.19  (3637/20000)